In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#       print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_dir = '/kaggle/input/inaturalist-insects'  

# Transformations recommandées, appliquées uniquement à l'entraînement
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),        # Rotation aléatoire jusqu'à 30 degrés
    transforms.RandomHorizontalFlip(),    # Flip horizontal aléatoire
    transforms.RandomVerticalFlip(),      # Flip vertical aléatoire
    transforms.RandomResizedCrop(224),    # Recadrage aléatoire et redimensionnement à 224x224
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Ajustements couleurs
    transforms.ToTensor(),                 # Conversion en tenseur PyTorch
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalisation ImageNet
])

# Transformations pour validation/test (pas d'augmentation)
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Chargement datasets
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=train_transforms)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=val_transforms)

# DataLoaders pour chargement batch
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Exemple pour vérifier les classes et dataset size
print(f'Nombre de classes: {len(train_dataset.classes)}')
print(f'Taille train dataset: {len(train_dataset)}')
print(f'Taille val dataset: {len(val_dataset)}')
